# BO runs

In [1]:
import torch
from botorch.models import FixedNoiseGP, SingleTaskGP
from gpytorch.kernels import ScaleKernel
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from botorch.acquisition.analytic import ExpectedImprovement
import matplotlib.pyplot as plt
import numpy as np
import pickle
import sys
import time

In /home/cokes/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/cokes/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/cokes/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/cokes/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/cokes/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylel

In [2]:
which_acquisition = "EI"
# which_acquisition = "max y_hat"
# which_acquisition = "max sigma"

load data from `prepare_Xy.ipynb`

In [3]:
X = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['X']
y = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['y']
y = np.reshape(y, (np.size(y), 1)) # for the GP
nb_data = np.size(y)
nb_data

69839

convert to torch tensors

In [4]:
X = torch.from_numpy(X)
y = torch.from_numpy(y)

In [5]:
X.size()

torch.Size([69839, 12])

In [6]:
y.size()

torch.Size([69839, 1])

In [7]:
X_unsqueezed = X.unsqueeze(1)

In [8]:
nb_data

69839

number of COFs for initialization

In [9]:
nb_COFs_initialization = 10

In [10]:
def bo_run(nb_iterations, verbose=False):
    assert nb_iterations > nb_COFs_initialization
    
    # select initial COFs for training data randomly.
    # idea is to keep populating this ids_acquired and return it for analysis.
    ids_acquired = np.random.choice(np.arange((nb_data)), size=nb_COFs_initialization, replace=False)

    # initialize acquired y, since it requires normalization
    y_acquired = y[ids_acquired]
    # standardize outputs
    y_acquired = (y_acquired - torch.mean(y_acquired)) / torch.std(y_acquired)
    
    for i in range(nb_COFs_initialization, nb_iterations):
        print("iteration:", i, end="\r")
        # construct and fit GP model
        model = SingleTaskGP(X[ids_acquired, :], y_acquired)
        mll = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_model(mll)

        # set up acquisition function
        if which_acquisition == "EI":
            acquisition_function = ExpectedImprovement(model, best_f=y_acquired.max().item())
            
            # compute aquisition function at each COF in the database. 
#             batch_size = 35000 # need to do in batches to avoid mem issues
#             acquisition_values = torch.zeros((nb_data))
#             acquisition_values[:] = np.NaN # for safety
#             nb_batches = nb_data // batch_size
#             for ba in range(nb_batches+1):
#                 id_start = ba * batch_size
#                 id_end   = id_start + batch_size
#                 if id_end > nb_data:
#                     id_end = nb_data
#                 with torch.no_grad():
#                     acquisition_values[id_start:id_end] = acquisition_function.forward(X_unsqueezed[id_start:id_end])
#             assert acquisition_values.isnan().sum().item() == 0 # so that all are filled properly.
            with torch.no_grad(): # to avoid memory issues; we arent using the gradient...
                acquisition_values = acquisition_function.forward(X_unsqueezed) # runs out of memory
        elif which_acquisition == "max y_hat":
            with torch.no_grad():
                acquisition_values = model.posterior(X_unsqueezed).mean.squeeze()
        elif which_acquisition == "max sigma":
            with torch.no_grad():
                acquisition_values = model.posterior(X_unsqueezed).variance.squeeze()
        else:
            raise Exception("not a valid acquisition function")

        # select COF to acquire with maximal aquisition value, which is not in the acquired set already
        ids_sorted_by_aquisition = acquisition_values.argsort(descending=True)
        for id_max_aquisition_all in ids_sorted_by_aquisition:
            if not id_max_aquisition_all.item() in ids_acquired:
                id_max_aquisition = id_max_aquisition_all.item()
                break

        # acquire this COF
        ids_acquired = np.concatenate((ids_acquired, [id_max_aquisition]))
        assert np.size(ids_acquired) == i + 1

        # update y aquired; start over to normalize properly
        y_acquired = y[ids_acquired, :] # start over to normalize y properly
        y_acquired = (y_acquired - torch.mean(y_acquired)) / torch.std(y_acquired)
        
        if verbose:
            print("\tacquired COF", id_max_aquisition, "with y = ", y[id_max_aquisition].item())
            print("\tbest y acquired:", y[ids_acquired].max().item())
        
    assert np.size(ids_acquired) == nb_iterations
    return ids_acquired

`ids_acquired[r, i]` will give ID of COF acquired during iteration `i` from run `r`.

In [11]:
bo_res = dict()
bo_res['nb_runs']       = 100
bo_res['nb_iterations'] = 250
bo_res['ids_acquired'] = []
for r in range(bo_res['nb_runs']):
    print("\n\nRUN", r)
    t0 = time.time()
    ids_acquired = bo_run(bo_res['nb_iterations'])
    bo_res['ids_acquired'].append(ids_acquired)
    print("took time t = ", (time.time() - t0) / 60, "min")



RUN 0
took time t =  7.237610721588135 min


RUN 1
took time t =  7.30598794221878 min


RUN 2
took time t =  7.276894311110179 min


RUN 3
took time t =  7.1658613920211796 min


RUN 4
took time t =  7.263525919119517 min


RUN 5
took time t =  7.240882853666942 min


RUN 6
took time t =  7.337699552377065 min


RUN 7
took time t =  7.284728991985321 min


RUN 8
took time t =  7.325448171297709 min


RUN 9
took time t =  7.570803308486939 min


RUN 10
took time t =  7.266802322864533 min


RUN 11
took time t =  7.262774999936422 min


RUN 12
took time t =  7.127003467082977 min


RUN 13
took time t =  7.364107779661814 min


RUN 14
took time t =  7.3784677942593895 min


RUN 15
took time t =  7.253569904963175 min


RUN 16
took time t =  7.307559335231781 min


RUN 17
took time t =  7.451583731174469 min


RUN 18
took time t =  7.553135732809703 min


RUN 19
took time t =  7.296091318130493 min


RUN 20
took time t =  7.2720548470815025 min


RUN 21
took time t =  7.0214725454648335

/home/cokes/.local/lib/python3.6/site-packages/gpytorch/distributions/multivariate_normal.py:263: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-10.
  NumericalWarning,


took time t =  7.055034768581391 min


RUN 29
took time t =  7.425979002316793 min


RUN 30
took time t =  7.601202404499054 min


RUN 31
took time t =  7.254582150777181 min


RUN 32
took time t =  7.32819838921229 min


RUN 33
took time t =  7.508639041582743 min


RUN 34
took time t =  7.175657308101654 min


RUN 35
took time t =  7.168862398465475 min


RUN 36
took time t =  7.118225149313608 min


RUN 37
took time t =  7.218559682369232 min


RUN 38
took time t =  7.178347424666087 min


RUN 39
took time t =  7.149571132659912 min


RUN 40
took time t =  7.397369893391927 min


RUN 41
took time t =  7.374075202147166 min


RUN 42
took time t =  7.3341242591540015 min


RUN 43
took time t =  7.261066178480784 min


RUN 44
took time t =  7.141440697511038 min


RUN 45
took time t =  7.313784658908844 min


RUN 46
took time t =  7.267547098795573 min


RUN 47
took time t =  7.20398888985316 min


RUN 48
took time t =  7.437757857640585 min


RUN 49
took time t =  7.218947410583496 mi

/home/cokes/.local/lib/python3.6/site-packages/gpytorch/distributions/multivariate_normal.py:263: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-10.
  NumericalWarning,


took time t =  7.436510316530863 min


RUN 82
took time t =  7.304804150263468 min


RUN 83
took time t =  7.327514203389486 min


RUN 84
took time t =  7.376877991358439 min


RUN 85
took time t =  7.5617250045140585 min


RUN 86
took time t =  7.26043134133021 min


RUN 87
took time t =  7.4054002404212955 min


RUN 88
took time t =  7.160198378562927 min


RUN 89
took time t =  7.317469696203868 min


RUN 90
took time t =  7.337529293696085 min


RUN 91
took time t =  7.506365013122559 min


RUN 92
took time t =  7.476064924399058 min


RUN 93
took time t =  7.318663656711578 min


RUN 94
took time t =  7.3077609499295555 min


RUN 95
took time t =  8.31069944302241 min


RUN 96
took time t =  7.400353403886159 min


RUN 97
took time t =  7.146096642812093 min


RUN 98
took time t =  7.348718508084615 min


RUN 99
took time t =  7.142310913403829 min


In [12]:
with open('bo_results' + which_acquisition + '.pkl', 'wb') as file:
    pickle.dump(bo_res, file)